## Customer Churn Analytics - Model Training

We will be working with three tables that contains Customer data, Communications data and Order data for a Retail Store. These dataset can be used to understand the consumer behaviour to predict churn, in this example.

These datasets were generated for this demo using a Kaggle dataset below.

Reference: https://www.kaggle.com/uttamp/store-data


In this notebook, we will work with the CURATED_CUSTOMER_CHURN table that is created by the Feature Engineering notebook and train a classification model using XGBoost to predict customer churn.

In [13]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
import snowflake.snowpark.types as T
import preprocessing as pp
from snowflake.snowpark.functions import sproc, col
import json
import matplotlib.pyplot as plt
import io
import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

## 1. Create a Snowpark Session


In [14]:
with open('creds.json') as f:
        data = json.load(f)
        connection_parameters = {
          'account': data['account'],
          'user': data['user'],
          'password': data['password'], #getpass.getpass(),
          'schema': data['schema'],
          'database': data['database'],
          'warehouse': data['warehouse']}
session = Session.builder.configs(connection_parameters).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='BI_MEDIUM_WH', CURRENT_DATABASE()='SUMMIT_DB', CURRENT_SCHEMA()='CHURN')]


In [15]:
session.add_import('preprocessing')
session.add_packages('snowflake-snowpark-python','scipy', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'xgboost', 'cachetools')

The version of package pandas in the local environment is 1.5.2, which does not fit the criteria for the requirement pandas. Your UDF might not work when the package version is different between the server and your local environment
The version of package numpy in the local environment is 1.23.5, which does not fit the criteria for the requirement numpy. Your UDF might not work when the package version is different between the server and your local environment
The version of package xgboost in the local environment is 1.7.1, which does not fit the criteria for the requirement xgboost. Your UDF might not work when the package version is different between the server and your local environment


## 2. Create stages for models and udfs


In [16]:
query = "create or replace stage models" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"
        
session.sql(query).collect()

[Row(status='Stage area MODELS successfully created.')]

In [17]:
query = "create or replace stage udf" +\
        " copy_options = (on_error='skip_file')"
        
session.sql(query).collect()

[Row(status='Stage area UDF successfully created.')]

In [18]:
def save_file(session, model, path):
  input_stream = io.BytesIO()
  joblib.dump(model, input_stream)
  session._conn._cursor.upload_stream(input_stream, path)
  return "successfully created file: " + path

## 3. Feature Engineering and XGBoost Model Training in Python Stored Procedures


First, we will do feature engineering to add new features, drop features and apply one-hot encoding to categorical fields.

In [19]:
src_table_name = 'TRANSFORMED_CUSTOMER_CHURN'
tgt_table_name = 'CURATED_CUSTOMER_CHURN'

def preprocessData(dfRaw):
    dfTransformed = dfRaw.drop(dfRaw["CREATED_DT"],dfRaw["FIRST_ORDER_DT"],dfRaw["LAST_ORDER_DT"])
    encoder_input_cols = ["FAV_DELIVERY_DAY", "CITY"]
    ohe = pp.OneHotEncoder(input_cols=encoder_input_cols)
    ohe.fit(dfTransformed)
    ohe_dfTransformed = ohe.transform(dfTransformed)
    return ohe_dfTransformed

def dataPrep(session: Session) -> str:
    src_df = session.table(src_table_name)
    df=preprocessData(src_df)
    df.write.mode("overwrite").save_as_table(tgt_table_name)
    return 'SUCCESS'

# Create an instance of StoredProcedure using the sproc() function
data_prep_sp = sproc(dataPrep, replace=True)

SnowparkSessionException: (1409): More than one active session is detected. When you call function 'udf' or use decorator '@udf', you must specify the 'session' parameter if you created multiple sessions.Alternatively, you can use 'session.udf.register' to register UDFs

In [ ]:
data_prep_sp()

'SUCCESS'

![Original Data Frame](images/image1.png)

In [ ]:
#plot refill against customers retained
df_retained = session.table(tgt_table_name).select('CUSTOMER_ID','RETAINED','REFILL')\
             .group_by(col('RETAINED'), col('REFILL')) \
             .count() \
             .toPandas()

pd.pivot_table(df_retained,
              values='COUNT',
              index='RETAINED',
              columns='REFILL') \
             .plot.bar()

TypeError: no numeric data to plot

In [ ]:
def train_model(session: Session) -> str:
    snowdf = session.table(tgt_table_name).drop('CUSTOMER_ID')
    # split the train and test set
    snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82) # use seed to make the split repeatable
    
    # save the train and test sets as time stamped tables in Snowflake 
    snowdf_train.write.mode("overwrite").save_as_table(tgt_table_name + "_TRAIN")
    snowdf_test.write.mode("overwrite").save_as_table(tgt_table_name +"_TEST")
    
    # Imputing missing values in numeric columns
    numeric_types = [T.DecimalType, T.LongType, T.DoubleType, T.FloatType, T.IntegerType]
    numeric_columns = [c.name for c in snowdf.schema.fields if type(c.datatype) in numeric_types]
    categorical_columns = [c.name for c in snowdf.schema.fields if type(c.datatype) not in numeric_types]
    for column in numeric_columns:
        snowdf = snowdf.fillna(snowdf.select(F.median(column)).collect()[0][0], column)
        
    cust = snowdf_train.drop("RETAINED").to_pandas() # drop labels for training set
    cust_labels = snowdf_train.select("RETAINED").to_pandas()
    cust_test = snowdf_test.drop("RETAINED").to_pandas()
    cust_test_labels = snowdf_test.select("RETAINED").to_pandas()
  
    standardizer = StandardScaler()
    xgbc_classifier = XGBClassifier(
          colsample_bytree=0.6621869047211867,
          learning_rate=0.006344788828608473,
          max_depth=6,
          min_child_weight=8,
          n_estimators=1115,
          n_jobs=100,
          subsample=0.26613420034295865,
          verbosity=0,
          random_state=829929877,
          nthread=1
    )
    full_pipeline = Pipeline([
            ('standardizer', standardizer),
            ('classfier', xgbc_classifier)
        ])

    # fit the preprocessing pipeline and the model together
    full_pipeline.fit(cust, cust_labels)
    y_pred = full_pipeline.predict_proba(cust_test)[:,1]
    predictions = [round(value) for value in y_pred]
    balanced_accuracy =  balanced_accuracy_score(cust_test_labels, predictions)
    print("Model testing completed.\n   - Model Balanced Accuracy: %.2f%%" % (balanced_accuracy * 100.0))
    
    #Confusion Matrix
    cm = confusion_matrix(cust_test_labels, predictions)
    TN, FP, FN, TP = confusion_matrix(cust_test_labels, predictions).ravel()
    accuracy =  (TP+TN) /(TP+FP+TN+FN)
   
    # save the full pipeline including the model
    save_file(session, full_pipeline, "@MODELS/cust_churn.joblib")
    return accuracy

# Create an instance of StoredProcedure using the sproc() function
train_model_sp = sproc(train_model, replace=True)

In [ ]:
train_model_sp()

Failed to execute query [queryID: 01a90c2b-0504-3574-0035-cd870003a516] CALL "SUMMIT_DB"."CHURN".SNOWPARK_TEMP_PROCEDURE_FPBXI1IO13()
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "/var/folders/7_/dz3yj_1n1rqbtq3n6qnnktsh0000gn/T/ipykernel_7359/2235441279.py", line 41, in train_model
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/sklearn/pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/sklearn/pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/usr/li

SnowparkSQLException: (1304): 01a90c2b-0504-3574-0035-cd870003a516: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "/var/folders/7_/dz3yj_1n1rqbtq3n6qnnktsh0000gn/T/ipykernel_7359/2235441279.py", line 41, in train_model
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/sklearn/pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/sklearn/pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c5091900e5b514db9c/lib/python3.8/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/usr/lib/python_udf/418f2eb2da12ecb131fa7f40d8988ff00193ef395077c7c509190

## 4. Model Inference - Creating a predict() UDF


In [ ]:
import sys
import cachetools
import os
from snowflake.snowpark.functions import udf, pandas_udf
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
session.add_import("@MODELS/cust_churn.joblib")  

@cachetools.cached(cache={})
def read_file(filename):
       import_dir = sys._xoptions.get("snowflake_import_directory")
       if import_dir:
              with open(os.path.join(import_dir, filename), 'rb') as file:
                     m = joblib.load(file)
                     return m

features = ['ESENT', 'EOPENRATE', 'ECLICKRATE', 'PAPERLESS', 'AVG_ORDER', 'DOOR_DELIVERY', 'REFILL', 'DIFF_BETWEEN_LAST_FIRST_DAYS',
              'DIFF_BETWEEN_FIRST_CREATED_DAYS', 'FAV_DELIVERY_DAY_FRIDAY', 'FAV_DELIVERY_DAY_MONDAY', 'FAV_DELIVERY_DAY_SATURDAY', 'FAV_DELIVERY_DAY_SUNDAY', 'FAV_DELIVERY_DAY_THURSDAY',
              'FAV_DELIVERY_DAY_TUESDAY','FAV_DELIVERY_DAY_WEDNESDAY', 'CITY_AUSTIN', 'CITY_DALLAS', 'CITY_HOUSTON', 'CITY_WACO' ]

@pandas_udf(name="batch_predict", is_permanent=True, stage_location="@udf", replace=True, max_batch_size=100)
def predict_batch(df: T.PandasDataFrame[float, float, float, float,
                                          float, int,  int, int, int, int, int, int, int, int, int, int,
                                          int, int, int, int]) -> T.PandasSeries[int]:
       m = read_file('cust_churn.joblib') 
       df.columns = features
       return m.predict(df)

: 

## 5. Create a prediction table


In [ ]:
from snowflake.snowpark import functions as F

pred_table_name="CHURN_PREDICTIONS"

snowdf_test = session.table(tgt_table_name).drop('CUSTOMER_ID')
inputs = snowdf_test.drop("RETAINED")
snowdf_results = snowdf_test.select(*inputs,
                    predict_batch(*inputs).alias('PREDICTION'), 
                    (F.col('RETAINED')).alias('ACTUAL_LABEL')
                    )

snowdf_results.write.mode("overwrite").save_as_table(pred_table_name)

snowdf_results.to_pandas().head(20)

: 

: 